In [ ]:
# todo NOTE: this script does not reproduce original result since we removed device IDs
# This script extract unique IoT devices from manually identified  and gpt identified devices.

import pandas as pd
import numpy as np

# input output file locations 
file_gpt_identified_device = '../../Device Identification/Device Category Mapping/gpt_device_id_generic_category.csv'
file_manually_identified_device = '../../Device Identification/Device Category Mapping/manual_device_category_merged.csv'

# read generic category mapped files
manual_device_category_merged = pd.read_csv(file_manually_identified_device).rename(columns={'generic_category':'man_generic_category'})
gpt_device_category_merged = pd.read_csv(file_gpt_identified_device)[['device_id', 'gpt_clean_vendor', 'gpt_clean_type','generic_category']].rename(columns={'generic_category':'gpt_generic_category'})
 
# retrieve common device both identified by gpt and manually
common_device = gpt_device_category_merged[gpt_device_category_merged.device_id.isin(manual_device_category_merged.device_id) == True]
manual_label_common = manual_device_category_merged[manual_device_category_merged.device_id.isin(common_device.device_id) == True][['device_id', 'vendor_name', 'man_generic_category', 'inspected']]
# todo NOTE: the following line cause runtime error since we removed device IDS 
common_device_merge = common_device.merge(manual_label_common, on='device_id', how='inner')
common_device_merge['vendor_name'] = common_device_merge['vendor_name'].str.lower().str.replace('-', '').str.replace(' ', '')
common_device_merge['gpt_clean_vendor'] = common_device_merge['gpt_clean_vendor'].str.lower().str.replace('-', '').str.replace(' ', '')
common_device_merge.dropna(inplace=True)
common_device_merge['gpt_clean_vendor'] = common_device_merge['gpt_clean_vendor'].replace({'ring':'amazon', 'nest':'google', 'idev':'idevices',
                                                                                           'yeelight':'xiaomi', 'logi':'logitech',
                                                                                           'raspberrypi':'raspberry', 'nanoleaf':'aurora',
                                                                                           'idevice':'idevices', 'dropcam':'google',
                                                                                           'number':'sleepnumber', 'marantz':'denon',
                                                                                           'wd':'westerndigital', 'withings':'nokia',
                                                                                           'smartthings':'samsung', 'myq':'chamberlain'})
common_device_merge['vendor_name'] = common_device_merge['vendor_name'].replace({'ring':'amazon', 'nest':'google',
                                                                                                   'idev':'idevices',
                                                                                                   'yeelight':'xiaomi', 'logi':'logitech',
                                                                                                   'raspberrypi':'raspberry', 'nanoleaf':'aurora',
                                                                                                   'idevice':'idevices', 'dropcam':'google',
                                                                                                   'number':'sleepnumber', 'marantz':'denon',
                                                                                                   'wd':'westerndigital', 'withings':'nokia',
                                                                                                   'smartthings':'samsung', 'myq':'chamberlain'})
common_device_merge['validate'] = np.where((common_device_merge['gpt_clean_vendor']==common_device_merge['vendor_name']), True, False)


# This code block is used to find the mismatched label between gpt identified labeling and manually identified labellings
# mismatched_df = common_device_merge[(common_device_merge['validate']==False) & (common_device_merge['inspected']==1)]
# mismatched_df = mismatched_df.groupby(['gpt_clean_vendor', 'gpt_generic_category', 'vendor_name', 'man_generic_category'])['device_id'].count().reset_index()
# mismatched_df = mismatched_df.rename(columns={'device_id': 'device_count'})
# mismatched_df = mismatched_df.sort_values(by='device_count', ascending=False)
# print(mismatched_df)



# common devices from gpt identified devices and manually identified devices
iot_list = ['Media/TV',
            'Home Appliance ',
            'Generic IoT',
            'Surveillance',
            'Home Automation',
            'Work Appliance',
            'Voice Assistant',
            'Game Console',
            'Vehicle']

common_devices_1st = common_device_merge[common_device_merge['validate']==True]   ## 1st part
common_devices_1st = common_devices_1st[['device_id', 'gpt_clean_vendor', 'gpt_generic_category', 'vendor_name', 'man_generic_category']]

common_devices_1st = common_devices_1st[(common_devices_1st.gpt_generic_category!='Computer')
                                        & (common_devices_1st.gpt_generic_category!='Router')
                                        & (common_devices_1st.gpt_generic_category!='Mobile Device ')
                                        & (common_devices_1st.gpt_generic_category!='Storage')
                                        & (common_devices_1st.gpt_generic_category!='Wearable')
                                        & (common_devices_1st.gpt_generic_category!='Other')]

common_devices_1st = common_devices_1st[(common_devices_1st.man_generic_category!='Computer')
                                        & (common_devices_1st.man_generic_category!='Router')
                                        & (common_devices_1st.man_generic_category!='Mobile Device ')
                                        & (common_devices_1st.man_generic_category!='Storage')
                                        & (common_devices_1st.man_generic_category!='Wearable')
                                        & (common_devices_1st.man_generic_category!='Other')]



# manual identified devices not in gpt
manual_not_in_gpt_2nd = manual_device_category_merged[(manual_device_category_merged.device_id.isin(gpt_device_category_merged.device_id)==False)
                                                      & (manual_device_category_merged.inspected==1)]
manual_not_in_gpt_2nd = manual_not_in_gpt_2nd[['device_id', 'vendor_name', 'man_generic_category']]
manual_not_in_gpt_2nd = manual_not_in_gpt_2nd[(manual_not_in_gpt_2nd.man_generic_category!='Computer')
                                              & (manual_not_in_gpt_2nd.man_generic_category!='Router')
                                              & (manual_not_in_gpt_2nd.man_generic_category!='Mobile Device ')
                                              & (manual_not_in_gpt_2nd.man_generic_category!='Storage')
                                              & (manual_not_in_gpt_2nd.man_generic_category!='Wearable')
                                              & (manual_not_in_gpt_2nd.man_generic_category!='Other')]


# gpt identified devices not in manual
gpt_not_in_manual_3rd = gpt_device_category_merged[gpt_device_category_merged.device_id.isin(manual_device_category_merged.device_id) == False]
gpt_not_in_manual_3rd = gpt_not_in_manual_3rd[['device_id', 'gpt_clean_vendor', 'gpt_generic_category']]
gpt_not_in_manual_3rd = gpt_not_in_manual_3rd[(gpt_not_in_manual_3rd.gpt_generic_category!='Computer')
                                              & (gpt_not_in_manual_3rd.gpt_generic_category!='Router')
                                              & (gpt_not_in_manual_3rd.gpt_generic_category!='Mobile Device ')
                                              & (gpt_not_in_manual_3rd.gpt_generic_category!='Storage')
                                              & (gpt_not_in_manual_3rd.gpt_generic_category!='Wearable')
                                              & (gpt_not_in_manual_3rd.gpt_generic_category!='Other')]


# combine all iot devices
combined_iot_devices = pd.concat([common_devices_1st, manual_not_in_gpt_2nd, gpt_not_in_manual_3rd])

# todo save result to file
# file_target_IoT_devices = '../../Device Identification/IoT_devices_v5.csv'
# combined_iot_devices.to_csv(file_target_IoT_devices, index=False)